# 🔭 UFO Sightings & Weather Analysis

This notebook aims to enrich a dataset of UFO sightings with historical weather information using the Open-Meteo API.

### Main objectives:
- Clean and prepare the sightings data (location, date, time)
- Retrieve daily weather data for each sighting
- Use caching to avoid repeated API calls
- Merge weather data back into the sightings dataset
- Export the final dataset for analysis and visualization

The project is structured for easy reuse and sharing, with all functions separated into a Python module (`weather_utils.py`) and a clean, well-documented notebook.

In [1]:
# ============================================
# 🔭 UFO Forecast Analysis — Notebook 1
# ============================================

import pandas as pd
from ufo_weather_utils import (
    build_weather_cache_from_df,
    attach_weather_to_df
)

# 1️⃣ LOAD AND PREPARE THE DATASET
url = "https://raw.githubusercontent.com/martin-paz-y/ufo_project/main/split_1.csv"
ufo_df = pd.read_csv(url)
ufo_df.columns = ufo_df.columns.str.strip()

ufo_df["date"] = ufo_df["datetime"].str.split(" ").str[0].str.replace('/', '-')
ufo_df["time"] = ufo_df["datetime"].str.split(" ").str[1]
ufo_df["date"] = pd.to_datetime(ufo_df["date"], format='%m-%d-%Y', errors='coerce').dt.strftime('%Y-%m-%d')

ufo_df.loc[ufo_df["latitude"] == "33q.200088", "latitude"] = "33.200088"
ufo_df["latitude"] = pd.to_numeric(ufo_df["latitude"], errors='coerce').round(2)
ufo_df["longitude"] = pd.to_numeric(ufo_df["longitude"], errors='coerce').round(2)

ufo_df = ufo_df.dropna(subset=["latitude", "longitude", "date"]).reset_index(drop=True)

print("✅ Cleaned dataset — first 5 rows:")
print(ufo_df.head(5))

# 2️⃣ PROCESS FULL DATASET
cache = build_weather_cache_from_df(ufo_df, delay=0.5)
ufo_weather_full = attach_weather_to_df(ufo_df, cache)

print("\n✅ UFO dataset enriched with weather data:")
print(ufo_weather_full.head(10))

ufo_weather_full.to_csv("ufo_weather_full.csv", index=False)
print("\n💾 File saved: ufo_weather_full.csv")

C:\Users\tozes\AppData\Local\Temp\ipykernel_7664\111828603.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '33.200088' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ufo_df.loc[ufo_df["latitude"] == "33q.200088", "latitude"] = "33.200088"


✅ Cleaned dataset — first 5 rows:
           datetime        date   time          city state country     shape  \
0  10/10/2007 01:00  2007-10-10  01:00       lebanon    or      us     light   
1  10/10/2007 01:00  2007-10-10  01:00   stockbridge    ga      us  changing   
2  10/10/2007 04:00  2007-10-10  04:00        denver    co      us  changing   
3  10/10/2007 04:30  2007-10-10  04:30  jacksonville    fl      us    sphere   
4  10/10/2007 06:00  2007-10-10  06:00         indio    ca      us      oval   

   duration (seconds) duration (hours/min)  \
0             14400.0              4 hours   
1              3600.0               1 hour   
2              2700.0          45 minutes+   
3                30.0           30 seconds   
4               600.0              10 min.   

                                            comments  latitude  longitude  \
0  Small Orange-White &quot;star&quot; that moves...     44.54    -122.91   
1                                     i was abducted  

In [2]:
display(ufo_weather_full)

,datetime,date,time,city,state,country,shape,duration (seconds),duration (hours/min),comments,latitude,longitude,lighting_conditions,temp_max,temp_min,precipitation_sum,weather_code
0,10/10/2007 01:00,2007-10-10,01:00,lebanon,or,us,light,14400.0,4 hours,Small Orange-White &quot;star&quot; that moves...,44.54,-122.91,night,14.5,8.4,9.8,61.0
1,10/10/2007 01:00,2007-10-10,01:00,stockbridge,ga,us,changing,3600.0,1 hour,i was abducted,33.54,-84.23,night,28.1,18.4,0.0,1.0
2,10/10/2007 04:00,2007-10-10,04:00,denver,co,us,changing,2700.0,45 minutes+,Huge lighted cluster in the eastern sky. ((NU...,39.74,-104.98,night,24.6,5.0,0.0,2.0
3,10/10/2007 04:30,2007-10-10,04:30,jacksonville,fl,us,sphere,30.0,30 seconds,half mile from de saint Jhons River&#44 low fl...,30.33,-81.66,night,30.1,21.0,0.0,3.0
4,10/10/2007 06:00,2007-10-10,06:00,indio,ca,us,oval,600.0,10 min.,Observed 2 white clouds of identical shape in ...,33.72,-116.21,daylight,32.0,14.8,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9913,12/23/2012 07:30,2012-12-23,07:30,appleton,wi,us,fireball,300.0,3-5 minutes,5-6 fireball shaped object floting across the sky,44.26,-88.42,daylight,-5.6,-10.7,0.0,3.0
9914,12/23/2012 14:20,2012-12-23,14:20,jaipur (india),NaN,NaN,fireball,120.0,2 minutes,We believe it was a UFO as it was a very brig...,26.91,75.79,daylight,21.4,6.0,0.0,0.0
9915,12/23/2012 15:00,2012-12-23,15:00,opp,al,us,light,5.0,5 seconds,Brief star like light to the left of sun,31.28,-86.26,daylight,13.9,2.4,0.0,3.0
9916,12/23/2012 15:45,2012-12-23,15:45,troy,nh,us,disk,600.0,10 minutes,Troy NH disk shaped ufo sighting overlooking c...,42.82,-72.18,daylight,0.7,-5.2,0.0,3.0
